In [2]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import os
from dynamic_object_detection.params import *
from dynamic_object_detection.process_KITTI import * 
import yaml
import robotdatapy as rdp

time_tol = 1.0 # s
vel_estimate_dt = 0.25 # s
detected_threshold = 1.0 # m

ModuleNotFoundError: No module named 'dynamic_object_detection'

In [2]:
os.environ['BAG_PATH'] = os.path.expanduser('/home/${USER}/Downloads/hamilton_cv_gt_2025-05-11.bag')
gt_bag = '~/Downloads/husky_gt/'

gt_topic = lambda obj: f'/{obj}/world'
cam = 'Husky'
objs = ['BD01', 'SCOUT2']

In [3]:
def project(obj_pos, cam_info):
    obj_pixel = cam_info['K'] @ obj_pos
    obj_pixel /= obj_pixel[2]
    obj_pixel = obj_pixel[:2]

    return obj_pixel

def within_cam_view(T_cam_obj, cam_info):
    obj_pos = T_cam_obj[:3, 3]

    if obj_pos[2] <= 0: return False

    obj_pixel = project(obj_pos, cam_info)

    return 0 <= obj_pixel[0] <= cam_info['W'] and \
           0 <= obj_pixel[1] <= cam_info['H']

def estimate_vel(pose_data, t, dt):
    idx1 = pose_data.idx(t - dt, force_single=True)
    idx2 = pose_data.idx(t + dt, force_single=True)

    if idx1 == idx2: return None

    t1 = pose_data.times[idx1]
    t2 = pose_data.times[idx2]
    pose1 = pose_data.pose(t1)
    pose2 = pose_data.pose(t2)

    return np.linalg.norm(pose2[:3, 3] - pose1[:3, 3]) / (t2 - t1)

def get_min_vel(params, z):
    return params.tracking_params.min_vel_threshold + z * params.tracking_params.vel_threshold_gain

def mahalanobis_distance(detected_obj_pos, detected_obj_cov, obj_pos):
    diff = detected_obj_pos - obj_pos
    return np.sqrt(diff.T @ np.linalg.inv(detected_obj_cov) @ diff)

In [ ]:
def eval_run(run, base, seq):

    # ----------------------------
    # Load results and params
    # ---------------------------
    results_path = os.path.join('..', f'{run}.pkl')
    params_path = os.path.join('..', f'{run}.yaml')

    with open(results_path, 'rb') as f:
        results = pickle.load(f)

    params = Params.from_yaml(params_path)

    # ----------------------------
    # Load KITTI ground truth
    # ----------------------------
    # Our helpers (already defined in your KITTI loader file):
    #   - load_oxts, oxts_to_poses
    #   - load_labels
    #   - load_calib, load_image_paths, disparity_to_depth, stereo_matcher

    oxts_file = f"{base}/data_tracking_oxts/training/oxts/{seq}.txt"
    label_file = f"{base}/data_tracking_label_2/training/label_02/{seq}.txt"
    calib_file = f"{base}/calib/training/calib/{seq}.txt"
    left_dir   = f"{base}/data_tracking_image_2/training/image_02/{seq}"
    right_dir  = f"{base}/data_tracking_image_3/training/image_03/{seq}"

    calib = load_calib(calib_file)
    oxts = load_oxts(oxts_file)
    cam_poses = oxts_to_poses(oxts)        # list of 4x4 camera poses in ENU
    labels = load_labels(label_file)       # dict frame_idx -> list of object dicts
    left_paths = load_image_paths(left_dir)
    right_paths = load_image_paths(right_dir)
    matcher = stereo_matcher()

    # ----------------------------
    # Evaluation bookkeeping
    # ----------------------------
    cam_info = results['camera_info']
    total_should_have_detected_objects = 0
    total_detected_objects = 0
    total_objects = 0
    total_correct_detections = 0
    total_detected_error = 0
    total_detected_error_count = 0
    frames_without_detections = [0, 0]
    max_frames_without_detections = 0

    def did_not_detect(ind):
        frames_without_detections[ind] += 1
        nonlocal max_frames_without_detections
        max_frames_without_detections = max(max_frames_without_detections,
                                            frames_without_detections[ind])

    def detected(ind):
        frames_without_detections[ind] = 0

    # ----------------------------
    # Main loop over frames
    # ----------------------------
    for frame, time in enumerate(results['times']):
        try:
            cam_pose = cam_poses[frame]

            # stereo depth (optional – skip if not needed in eval)
            left, right = load_stereo_pair(left_paths[frame], right_paths[frame])
            disp = matcher.compute(left, right).astype(np.float32) / 16.0
            depth = disparity_to_depth(disp, calib['P2'], calib['P3'])

            frame_objs = results['objects'][frame]
            total_objects += len(frame_objs)
            correct_detection = [False] * len(frame_objs)

            if frame not in labels:
                continue

            for obj in labels[frame]:
                if obj["type"] in ["DontCare", "Misc"]:
                    continue

                # Convert object center to ENU
                T_obj_enu = object_cam_to_enu(obj, cam_pose, calib)
                Z = T_obj_enu[2, 3]

                if Z > params.depth_data_params.max_depth:
                    continue
                if not within_cam_view(T_obj_enu, cam_info):
                    continue

                # velocity estimation could be added here using consecutive frames
                estimate_vel_from_labels(labels, obj['track_id'], poses, dt=0.1)
                estimated_vel = 1.0  # placeholder

                if estimated_vel >= get_min_vel(params, Z):
                    false_negative = True

                    for j, detected_obj in enumerate(frame_objs):
                        if np.linalg.norm(detected_obj['point'] - T_obj_enu[:3, 3]) < detected_threshold:
                            false_negative = False
                            correct_detection[j] = True
                            total_detected_error += np.linalg.norm(
                                detected_obj['point'] - T_obj_enu[:3, 3])
                            total_detected_error_count += 1

                    if false_negative:
                        total_should_have_detected_objects += 1
                        did_not_detect(0)
                    else:
                        total_detected_objects += 1
                        detected(0)

            total_correct_detections += sum(correct_detection)

        except Exception as e:
            print(f"Skipping frame {frame} due to error: {e}")
            continue

    # ----------------------------
    # Report
    # ----------------------------
    print(f'Runtime: {results["runtime"]}')
    det_rate = total_detected_objects / max(1, (total_detected_objects + total_should_have_detected_objects))
    print(f'Out of {total_should_have_detected_objects + total_detected_objects} dynamic objects in frames, '
          f'{total_detected_objects} were detected: {det_rate * 100:.2f}%. {(1- det_rate) * 100:.2f}% false negatives.')
    correct_rate = total_correct_detections / max(1, total_objects)
    print(f'Out of {total_objects} detected dynamic objects, {total_correct_detections} were correct detections: '
          f'{correct_rate * 100:.2f}%. {(1 - correct_rate) * 100:.2f}% false positives.')
    if total_detected_error_count > 0:
        print(f'Average error {total_detected_error / total_detected_error_count} m')
    print(f'Max frames without detection: {max_frames_without_detections}')


In [ ]:
skip_runs = [f'out/skip_frames/hamilton_skip{n}' for n in (2, 3, 5)]
raft_runs = [f'out/raft/hamilton_{model}' for model in ('kitti', 'things', 'sintel', 'chairs')]
raft_12_runs = [f'out/raft/hamilton_{model}_12' for model in ('kitti', 'things', 'sintel', 'chairs')]
ablation_runs = [f'out/ablations/no_{thing}' for thing in ('morph', 'stddev_prune', 'threshold_gain')]
depth_runs = [f'out/depths/{depth}' for depth in ('6', '7-5', '10', '15')]
test_3d_runs = ['out/3d/3d']

runs = skip_runs + raft_runs + raft_12_runs + ablation_runs + depth_runs + test_3d_runs

runs = [f'out/raft/kitti_kitti']
base = '/home/jrached/cv_project_code/project/data/KITTI_tracking/unzipped'    
seq = "0003"

print(f'\nRunning sequence: {seq}')
for run in runs:
    print(f'\n{run}\n')
    eval_run(run, base, seq)


out/skip_frames/hamilton_skip2

Runtime: {'avg_batch_time': 2.1666026211554006, 'avg_frame_time': 0.09027510921480836, 'total_time': 268.65872502326965}
Out of 1283 dynamic objects in frames, 1088 were detected: 84.80%. 15.20% false negatives.
Out of 1384 detected dynamic objects, 1114 were correct detections: 80.49%. 19.51% false positives.
Average error 0.25054482210144874 m
Max frames without detection: 23

out/skip_frames/hamilton_skip3

Runtime: {'avg_batch_time': 2.5469597420060492, 'avg_frame_time': 0.10612332258358538, 'total_time': 211.39765858650208}
Out of 849 dynamic objects in frames, 715 were detected: 84.22%. 15.78% false negatives.
Out of 900 detected dynamic objects, 731 were correct detections: 81.22%. 18.78% false positives.
Average error 0.2518413168628891 m
Max frames without detection: 15

out/skip_frames/hamilton_skip5

Runtime: {'avg_batch_time': 2.539337224960327, 'avg_frame_time': 0.10580571770668029, 'total_time': 126.96686124801636}
Out of 513 dynamic objec